In [1]:
import gym_examples
import numpy as np
import gym

In [2]:
env = gym.make('gym_examples/GridWorld-v3', render_mode="human", size=3, \
    rewards=np.array([[1,2,3],[4,5,6],[7,8,9]]), costs=np.array([[1,2,3],[4,5,6],[7,8,9]]))
observation, info = env.reset()
for _ in range(100):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward_cost, terminated, truncated, info = env.step(action)
    print(observation, reward_cost, terminated, truncated, info)

    if terminated or truncated:
        observation, info = env.reset()
        
env.close()

/Users/noahliniger/opt/miniconda3/envs/FORL/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/noahliniger/opt/miniconda3/envs/FORL/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:252: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'list'>
  logger.warn(


{'agent': array([2, 0])} [7, 7] False False {}
{'agent': array([2, 0])} [7, 7] False False {}
{'agent': array([2, 0])} [7, 7] False False {}
{'agent': array([2, 2])} [9, 9] False False {}
{'agent': array([2, 2])} [9, 9] False False {}
{'agent': array([0, 2])} [3, 3] False False {}
{'agent': array([2, 2])} [9, 9] False False {}
{'agent': array([2, 2])} [9, 9] False False {}
{'agent': array([2, 0])} [7, 7] False False {}
{'agent': array([2, 0])} [7, 7] False False {}
{'agent': array([2, 0])} [7, 7] False False {}
{'agent': array([2, 0])} [7, 7] False False {}
{'agent': array([2, 0])} [7, 7] False False {}
{'agent': array([2, 2])} [9, 9] False False {}
{'agent': array([2, 2])} [9, 9] False False {}
{'agent': array([2, 0])} [7, 7] False False {}
{'agent': array([2, 0])} [7, 7] False False {}
{'agent': array([2, 0])} [7, 7] False False {}
{'agent': array([2, 2])} [9, 9] False False {}
{'agent': array([2, 0])} [7, 7] False False {}
{'agent': array([2, 0])} [7, 7] False False {}
{'agent': arr

In [11]:
import numpy as np

In [3]:
#Test on the constraint walls: sturcture: [[x1,y1,x2,y2],[same]]
A = np.array([[1,1,2,1],[1,2,2,2]])
location = np.array([1,1])
direction = np.array([1,0])
for constraint in A:
    A = np.array_equal(constraint[0:2], location)
    B = np.array_equal(constraint[2:5], location + direction)
    C = np.array_equal(constraint[2:5], location)
    D = np.array_equal(constraint[0:2], location + direction)
    A_and_B = A and B
    C_and_D = C and D
    test = A_and_B or C_and_D
    print(A,B,C,D,A_and_B,C_and_D, "Test:", test)
    if (np.array_equal(constraint[0:2], location)
        and np.array_equal(constraint[2:5], location + direction))\
            or (np.array_equal(constraint[2:5], location) \
                and np.array_equal(constraint[0:2], location + direction)):
        location = location
        break
    else:
        location = location + direction
print(location)

True True False False True False Test: True
[1 1]


In [36]:
A = np.array([])


In [19]:
print(A.value())

AttributeError: 'numpy.ndarray' object has no attribute 'value'